In [ ]:
!pip install konlpy
!pip install googletrans
!gensim

In [2]:
from xgboost import XGBClassifier
import xgboost
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import lightgbm as lgb
import sklearn.metrics as mt
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score, classification_report


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

import pandas as pd
import warnings 
warnings.filterwarnings('ignore')

# Read Review Data
import pandas as pd
from tqdm import tqdm
import nltk
import re 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem.wordnet import WordNetLemmatizer
from tqdm import tqdm
import re 
import tqdm.notebook as tq

%matplotlib inline

from sklearn import svm
from keras.utils import get_file
import os
import gensim
import subprocess
import numpy as np
import random
import requests
import pandas as pd
from IPython.core.pylabtools import figsize
import csv
import matplotlib.pyplot as plt

import gensim.downloader as api

from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
from numpy.linalg import norm
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import konlpy
from konlpy.tag import Okt
from konlpy.tag import Kkma
import re

import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
tqdm.pandas()

from googletrans import Translator
translator = Translator()

model = api.load('glove-wiki-gigaword-200')

tqdm.pandas(position=0, leave=True)

nltk.download('wordnet')
nltk.download('vader_lexicon')

[==================================================] 100.0% 252.1/252.1MB downloaded


[nltk_data] Downloading package wordnet to /Users/jeong-
[nltk_data]     gyeonghui/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package vader_lexicon to /Users/jeong-
[nltk_data]     gyeonghui/nltk_data...


True

In [4]:
temp = pd.read_excel("data/최종일까4.xlsx")
im = pd.read_excel("data/1970_2019_imdb2.xlsx")
df = pd.read_excel("data/1970_2019_영화전체.xlsx")

In [5]:
# Data
# English Preprocessing ----------------------------------------------------------------
print("Start English Preprocessing...")
def trim_en(text):
    text = str(text)
    #text =text.lower()
    text=re.sub("[(<.*?>)'']"," ",text)
    text=re.sub("(\\W|\\d)"," ",text)
    text=text.strip()
    
    return text

lemm =  WordNetLemmatizer()
sa = SentimentIntensityAnalyzer()

im.리뷰 = im.리뷰.progress_apply(trim_en)
im.리뷰 = im.리뷰.progress_apply(lambda s:" ".join([lemm.lemmatize(w) for w in str(s).split()]) )


# Sentimental Analysis ----------------------------------------------------------------
print("Start  Sentimental Analysis...")
im['neg'] = 0
im['pos'] = 0
im['neu'] = 0
im['compound'] = 0

for i in tq.tqdm(range(len(im))):
    scores = sa.polarity_scores(str(im.loc[i, '리뷰']))
    im.loc[i,'neg'] = scores['neg']
    im.loc[i,'pos'] = scores['pos']
    im.loc[i,'neu'] = scores['neu']
    im.loc[i,'compound'] = scores['compound']

# Comment Analysis ----------------------------------------------------------------
print("Start  Comment Analysis...")
words = ["스토리", "음악", "연출", "배우","연기"]
translator = Translator()
trans = translator.translate(words, dest="en")

lst2 = [model.most_similar(positive=[lemm.lemmatize(word.text.lower())],topn=50) for word in trans ]
lst2 = [ [trans[k].text] + [j[0] for j in i]  for k,i in enumerate(lst2)]

values = []
values_pos = []
values_neg = []

for i in tq.tqdm(range(len(im))):
    review1 = im.loc[i, '리뷰']
    value = [0.001 for i in range(len(lst2))]
    value_pos = [0.001 for i in range(len(lst2))]
    value_neg = [0.001 for i in range(len(lst2))]
    string_split = str(review1).split()
    for k2, i in enumerate(string_split):
        for k,j in enumerate(lst2):
            if i in j:
                value[k] +=1
                left = max([0, k2-10])
                right = k2+10
                window = " ".join(string_split[left:right])
                scores = sa.polarity_scores(window)
                if scores['compound']>0:
                    value_pos[k] += scores['pos']
                if scores['compound']<0:
                    value_neg[k] += scores['neg']

    #Normalize  
    #v = [value_pos[i]/value[i] for i in range(len(value_pos))]
    #values_pos.append(v)
    # else
    values_pos.append(value_pos)


    #Normalize
    #v = [value_neg[i]/value[i] for i in range(len(value_neg))]
    #values_neg.append(v)
    values_neg.append(value_neg)

    #Normalize
    #SUM = sum(value)
    #v = [i/SUM for i in value]
    #values.append(v)
    values.append(value)
    
labels = words
temp1 = pd.DataFrame(values, columns = [i+"_com" for i in  labels])
temp2 = pd.DataFrame(values_pos, columns = [i+"_pos" for i in  labels])
temp3 = pd.DataFrame(values_neg, columns = [i+"_neg" for i in  labels])

data1 = pd.concat([im[['neg', 'pos','neu' ,'compound']], temp1, temp2, temp3],axis=1)
data1.info()
data1.head()

Start English Preprocessing...



Start  Sentimental Analysis...



Start  Comment Analysis...


NameError: name 'words2' is not defined

In [7]:
def cosim(x,y):
    denom = norm(x)* norm(y)
    if denom==0:
        return 0
    return np.dot(x,y.T)/(norm(x)* norm(y))

def nuclear_norm(x,y):
    return np.sum(np.abs(x-y))

def spectral_norm(x,y):
    return np.max(np.abs(x-y))

def l2_norm(x,y):
    return np.sqrt(np.sum((x-y)**2))

def rbf_kernel(x,y):
    return np.exp(-1/2*(np.sum((x-y)**2)))

def most_similar(X, x, n, method="cosine"):
    if method=="cosine":
        lst = [(cosim(X[x, :], X[i,:]), i) for i in range(len(X))]
        lst.sort(reverse=True)
    if method=="nuclear":
        lst = [(nuclear_norm(X[x, :], X[i,:]), i) for i in range(len(X))]
        lst.sort(reverse=False)
    if method=="spectral":
        lst = [(spectral_norm(X[x, :], X[i,:]), i) for i in range(len(X))]
        lst.sort(reverse=False)
    if method=="l2":
        lst = [(l2_norm(X[x, :], X[i,:]), i) for i in range(len(X))]
        lst.sort(reverse=False)    
    if method=="rbf":
        lst = [(rbf_kernel(X[x, :], X[i,:]), i) for i in range(len(X))]
        lst.sort(reverse=False)    
    return lst[:n]


# Korean 

okt = Okt()
kkma = Kkma()
def trim_ko(x):
    text=re.sub("[(<.*?>)/'…“”']"," ",x)
    #text=re.sub("(\\W|\\d)"," ",text)
    
    #text = kkma.sentences(text)
    #print(text)
    text = " ".join([w[0] for w in okt.pos(text) if w[1]=="Noun" or w[1]=='Verb' or w[1]=="Adjective"])
    #textx= " ".join([w[0] for w in kkma.pos(text) if w[1]=='NNG' or w[1]=='VA' or w[1]=="MAG"  or w[1]=='MAC'])

    #remove whitespace
    text=text.strip()
    return text


df.스토리_ko.fillna("", inplace=True)
df.스토리_ko = df.스토리_ko.astype(str)
df.genres = df.genres.astype(str)

df.스토리_ko = df.스토리_ko.apply(lambda x:x.replace("\n"," "))
df.스토리_ko = df.스토리_ko.progress_apply(trim_ko)

corpus = df.스토리_ko[(df.스토리_ko.notnull()) & (df.스토리_ko.apply(lambda x: True if len(x)>3 else False))]
corpus = corpus.reset_index()

vectorizer = TfidfVectorizer(max_features=2000)
X = vectorizer.fit_transform(corpus.스토리_ko.values)
X = X.toarray()

pca = PCA(n_components=5)
pca.fit(X)

new_X = pca.fit_transform(X)

kmeans = KMeans(n_clusters=10, random_state=0).fit(new_X)

data2 = pd.DataFrame([corpus['index'], new_X[:,0], new_X[:,1], new_X[:,2],new_X[:,3], new_X[:,4], kmeans.labels_]).T
data2.columns=['df_index','x0','x1','x2','x3','x4','label']
data2.df_index = data2.df_index.astype(int)
data2.label = data2.label.astype(int)

print("Kmeans...")
for i in tqdm(range(kmeans.cluster_centers_.shape[0])):
    data2['center_angle'+str(i+1)] = 0
    for j in range(len(data2)):
        data2.loc[j, 'center_angle'+str(i+1)] =  cosim(kmeans.cluster_centers_[i,:], data2.iloc[j, 1:kmeans.cluster_centers_.shape[1]+1].values )

data2 = data2.set_index('df_index')


Kmeans...


NameError: name 'data' is not defined

In [9]:
temp = temp[['index', 'movienm', 'showtm', 'prdtyear', 'domestic', 'mpaa', 'raters',
       'ratings', 'kor_revenue', 'kor_audience', 'sf', 'family', 'performance',
       'horror', 'etc', 'documentary', 'drama', 'romance', 'musical',
       'mystery', 'crime', 'history', 'western', 'adult', 'thriller',
       'animation', 'action', 'adventure', 'war', 'comedy', 'fantasy',
       'director_appearance', 'director_revenue', 'distributor_share',
       'opendt_quarter', 'year_gap', 'showtypes_num', 'actor_score']]

temp = temp.set_index('index')
temp = temp.join(data1)
temp = temp.join(data2)
temp

,movienm,showtm,prdtyear,domestic,mpaa,raters,ratings,kor_revenue,kor_audience,sf,...,center_angle1,center_angle2,center_angle3,center_angle4,center_angle5,center_angle6,center_angle7,center_angle8,center_angle9,center_angle10
index,,,,,,,,,,,,,,,,,,,,,
2,드라이브,100,2011,35061555,15,228016,7.2,156237980,D,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,조조 래빗,108,2019,33370906,12,250206,7.9,965083200,C,0,...,0.323093,-0.397164,-0.744790,0.569103,-0.155152,-0.116992,0.661112,0.204118,-0.224096,0.072463
10,존 윅 - 리로드,122,2017,92029184,18,351231,7.5,2231583601,C,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,메멘토,113,2000,25544867,15,1096788,8.4,325278300,D,0,...,0.204231,0.775064,0.311798,-0.040746,-0.791138,0.087187,-0.185423,-0.102267,0.199356,-0.904310
25,존 윅: 특별판,107,2014,43037835,18,519268,7.4,81322120,D,0,...,-0.526295,-0.083300,-0.435791,0.897787,-0.041482,0.062484,-0.422683,-0.503666,0.109327,0.159718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5780,쿼터매인,99,1986,3751699,12,8643,4.6,147000,F,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5783,크로커다일 던디2,112,1988,109306210,15,844,3.3,39000,F,0,...,0.356034,0.700931,0.724647,-0.735532,-0.386863,-0.007150,-0.099774,0.123897,0.083190,-0.655104
5788,탱고와 캐쉬,100,1989,63408614,15,307534,7.5,266000,F,0,...,0.127171,0.796803,0.808207,-0.543126,-0.196476,-0.188776,-0.566519,-0.312025,0.121394,-0.431946


In [10]:
temp.columns

Index(['movienm', 'showtm', 'prdtyear', 'domestic', 'mpaa', 'raters',
       'ratings', 'kor_revenue', 'kor_audience', 'sf', 'family', 'performance',
       'horror', 'etc', 'documentary', 'drama', 'romance', 'musical',
       'mystery', 'crime', 'history', 'western', 'adult', 'thriller',
       'animation', 'action', 'adventure', 'war', 'comedy', 'fantasy',
       'director_appearance', 'director_revenue', 'distributor_share',
       'opendt_quarter', 'year_gap', 'showtypes_num', 'actor_score', 'neg',
       'pos', 'neu', 'compound', '스토리_com', '음악_com', '연출_com', '배우_com',
       '연기_com', '스토리_pos', '음악_pos', '연출_pos', '배우_pos', '연기_pos', '스토리_neg',
       '음악_neg', '연출_neg', '배우_neg', '연기_neg', 'x0', 'x1', 'x2', 'x3', 'x4',
       'label', 'center_angle1', 'center_angle2', 'center_angle3',
       'center_angle4', 'center_angle5', 'center_angle6', 'center_angle7',
       'center_angle8', 'center_angle9', 'center_angle10'],
      dtype='object')

## 알고리즘 함수 정의

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

In [12]:
def DecisionTreeTest(x_train,x_test, y_train, y_test):
    clf = DecisionTreeClassifier()
    clf.fit(x_train,y_train)
    prediction=clf.predict(x_test)
    print("DecisionTreeTest 정확도 : %.4f"%accuracy_score(y_test,prediction))


def RandomForestTest(x_train,x_test, y_train, y_test):
    rf_clf = RandomForestClassifier(random_state=0)
    rf_clf.fit(x_train, y_train)
    pred = rf_clf.predict(x_test)
    accuracy=accuracy_score(y_test,pred)
    print('랜덤포레스트 정확도 : {0:.4f}'.format(accuracy))

def GBMTest(x_train,x_test, y_train, y_test) :
    #GBM모델 셋업
    gb_clf=GradientBoostingClassifier(random_state=0)
    gb_clf.fit(x_train,y_train)
    gb_pred=gb_clf.predict(x_test)
    gb_accuracy = accuracy_score(y_test,gb_pred)
    print('GradientBoostingClassifier 정확도:{0:.4f}'.format(gb_accuracy))
    
def LightGBMTest(x_train,x_test, y_train, y_test):
    lgbm_clf = LGBMClassifier(random_state=0)
    lgbm_clf.fit(x_train, y_train)
    pred = lgbm_clf.predict(x_test)
    accuracy=accuracy_score(y_test,pred)
    print('LGBMClassifier 정확도:{0:.4f}'.format(accuracy))

def MLPClassifierTest(x_train,x_test, y_train, y_test):
    MLP_clf = MLPClassifier()
    MLP_clf.fit(x_train,y_train)
    pred = MLP_clf.predict(x_test)
    accuracy=accuracy_score(y_test,pred)
    print('MLPClassifier 정확도:{0:.4f}'.format(accuracy))
#     print("MLPClassifier 수행시간 : {0:1.4f}초".format(time.time()-start_time))
    
    #http://blog.naver.com/PostView.nhn?blogId=samsjang&logNo=220969601609&categoryNo=0&parentCategoryNo=0&viewDate=&currentPage=1&postListTopCurrentPage=1&from=postView
    
def linearRegression(x_train,x_test, y_train, y_test) : 
    LR=LinearRegression()
    LR.fit(x_train,y_train)
    # y_test2 = LR.predict(x_test2)
    score=LR.score(x_test,y_test)
    print("선형회귀 값 : %.6f"%score)
#     print("Misclassification samples : %d" %(y_train2 != y_test2).sum())

def logisticRegression(x_train,x_test, y_train, y_test) : 
    LR=LogisticRegression(C=1000.0,random_state=0)
    LR.fit(x_train,y_train)
    # y_test2 = LR.predict(x_test2)
    score=LR.score(x_test,y_test)
    print("로지스틱 회귀 값 : %.6f"%score)
#     print("Misclassification samples : %d" %(y_train2 != y_test2).sum())

In [ ]:
pd.options.display.float_format = '{:.7f}'.format
#pd.set_option('display.float_format', None)

In [ ]:
temp.kor_audience.describe()

In [13]:
temp.columns

Index(['movienm', 'showtm', 'prdtyear', 'domestic', 'mpaa', 'raters',
       'ratings', 'kor_revenue', 'kor_audience', 'sf', 'family', 'performance',
       'horror', 'etc', 'documentary', 'drama', 'romance', 'musical',
       'mystery', 'crime', 'history', 'western', 'adult', 'thriller',
       'animation', 'action', 'adventure', 'war', 'comedy', 'fantasy',
       'director_appearance', 'director_revenue', 'distributor_share',
       'opendt_quarter', 'year_gap', 'showtypes_num', 'actor_score', 'neg',
       'pos', 'neu', 'compound', '스토리_com', '음악_com', '연출_com', '배우_com',
       '연기_com', '스토리_pos', '음악_pos', '연출_pos', '배우_pos', '연기_pos', '스토리_neg',
       '음악_neg', '연출_neg', '배우_neg', '연기_neg', 'x0', 'x1', 'x2', 'x3', 'x4',
       'label', 'center_angle1', 'center_angle2', 'center_angle3',
       'center_angle4', 'center_angle5', 'center_angle6', 'center_angle7',
       'center_angle8', 'center_angle9', 'center_angle10'],
      dtype='object')

In [14]:
temp.columns = ['movienm', 'showtm', 'prdtyear', 'domestic', 'mpaa', 'raters',
       'ratings', 'kor_revenue', 'kor_audience', 'sf', 'family', 'performance',
       'horror', 'etc', 'documentary', 'drama', 'romance', 'musical',
       'mystery', 'crime', 'history', 'western', 'adult', 'thriller',
       'animation', 'action', 'adventure', 'war', 'comedy', 'fantasy',
       'director_appearance', 'director_revenue', 'distributor_share',
       'opendt_quarter', 'year_gap', 'showtypes_num', 'actor_score', 'neg',
       'pos', 'neu', 'compound', 'story_com', 'music_com', 'direction_com',
       'actor_com', 'acting_com', 'story_pos', 'music_pos', 'direction_pos',
       'actor_pos', 'acting_pos', 'story_neg', 'music_neg', 'direction_neg',
       'actor_neg', 'acting_neg', 'x0', 'x1', 'x2', 'x3', 'x4',
       'label', 'center_angle1', 'center_angle2', 'center_angle3',
       'center_angle4', 'center_angle5', 'center_angle6', 'center_angle7',
       'center_angle8', 'center_angle9', 'center_angle10']

In [19]:
temp = temp.fillna(0)

In [23]:
호에 = temp[['actor_score', 'neg',
       'pos', 'neu', 'compound', 'story_com', 'music_com', 'direction_com',
       'actor_com', 'acting_com', 'story_pos', 'music_pos', 'direction_pos',
       'actor_pos', 'acting_pos', 'story_neg', 'music_neg', 'direction_neg',
       'actor_neg', 'acting_neg']]

In [64]:
temp.to_excel("최종일까_댓글빈도순.xlsx") # 이거 파일이 저장될테니 밑에 분석하실땐 이 파일 불러서 쓰시면 됩니다.

In [65]:
temp = temp[temp.compound != 0]

In [98]:
temp = temp[temp.compound != 1] #이건 제거해야합니다 compound 1인거 살려야합니다

In [24]:
호에

,actor_score,neg,pos,neu,compound,story_com,music_com,direction_com,actor_com,acting_com,story_pos,music_pos,direction_pos,actor_pos,acting_pos,story_neg,music_neg,direction_neg,actor_neg,acting_neg
index,,,,,,,,,,,,,,,,,,,,
2,52.00,0.100,0.196,0.704,0.9999,281.001,29.001,19.001,158.001,62.001,39.562,4.993,3.610,23.860,6.999,11.516,1.132,0.301,6.012,3.492
7,92.25,0.000,0.000,0.000,0.0000,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001
10,6.00,0.142,0.183,0.675,0.9998,505.001,12.001,11.001,209.001,67.001,60.987,1.521,2.413,24.304,5.455,39.561,1.431,0.211,18.843,8.441
13,32.00,0.111,0.152,0.737,0.9996,315.001,0.001,15.001,113.001,79.001,36.683,0.001,2.659,14.499,8.829,15.601,0.001,0.087,4.838,5.128
25,50.00,0.000,0.000,0.000,0.0000,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5780,0.00,0.000,0.000,1.000,0.0000,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001
5783,47.50,0.000,0.000,1.000,0.0000,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001
5788,0.00,0.000,0.000,1.000,0.0000,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001


In [99]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1236 entries, 2 to 3996
Data columns (total 72 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   movienm              1236 non-null   object 
 1   showtm               1236 non-null   int64  
 2   prdtyear             1236 non-null   int64  
 3   domestic             1236 non-null   int64  
 4   mpaa                 1236 non-null   int64  
 5   raters               1236 non-null   int64  
 6   ratings              1236 non-null   float64
 7   kor_revenue          1236 non-null   int64  
 8   kor_audience         1236 non-null   object 
 9   sf                   1236 non-null   int64  
 10  family               1236 non-null   int64  
 11  performance          1236 non-null   int64  
 12  horror               1236 non-null   int64  
 13  etc                  1236 non-null   int64  
 14  documentary          1236 non-null   int64  
 15  drama                1236 non-null   i

## 전체 데이터

In [132]:
X_data = temp[['showtm', 'prdtyear', 'domestic', 'mpaa', 'raters',
       'ratings', 'sf', 'family', 'performance',
       'horror', 'etc', 'documentary', 'drama', 'romance', 'musical',
       'mystery', 'crime', 'history', 'western', 'adult', 'thriller',
       'animation', 'action', 'adventure', 'war', 'comedy', 'fantasy',
       'director_appearance', 'director_revenue', 'distributor_share',
       'opendt_quarter', 'year_gap', 'showtypes_num', 'actor_score', 'neg',
       'pos', 'neu', 'compound', 'story_com', 'music_com', 'direction_com',
       'actor_com', 'acting_com', 'story_pos', 'music_pos', 'direction_pos',
       'actor_pos', 'acting_pos', 'story_neg', 'music_neg', 'direction_neg',
       'actor_neg', 'acting_neg', 'x0', 'x1', 'x2', 'x3', 'x4', 'label',
       'center_angle1', 'center_angle2', 'center_angle3', 'center_angle4',
       'center_angle5', 'center_angle6', 'center_angle7', 'center_angle8',
       'center_angle9', 'center_angle10']]
y_data = temp.kor_audience

In [133]:
X_train, X_test, y_train, y_test =\
train_test_split(X_data, y_data,\
                    test_size = 0.3, \
                    random_state = 78)

In [134]:
DecisionTreeTest(X_train, X_test, y_train, y_test)
RandomForestTest(X_train, X_test, y_train, y_test)
GBMTest(X_train, X_test, y_train, y_test)
MLPClassifierTest(X_train, X_test, y_train, y_test)
logisticRegression(X_train, X_test, y_train, y_test)
LightGBMTest(X_train, X_test, y_train, y_test)

DecisionTreeTest 정확도 : 0.5984
랜덤포레스트 정확도 : 0.6792
GradientBoostingClassifier 정확도:0.7035
MLPClassifier 정확도:0.4879
로지스틱 회귀 값 : 0.522911
LGBMClassifier 정확도:0.6954


## 스토리만 빼고

In [135]:
X_data = temp[['showtm', 'prdtyear', 'domestic', 'mpaa', 'raters',
       'ratings', 'sf', 'family', 'performance',
       'horror', 'etc', 'documentary', 'drama', 'romance', 'musical',
       'mystery', 'crime', 'history', 'western', 'adult', 'thriller',
       'animation', 'action', 'adventure', 'war', 'comedy', 'fantasy',
       'director_appearance', 'director_revenue', 'distributor_share',
       'opendt_quarter', 'year_gap', 'showtypes_num', 'actor_score', 'neg',
       'pos', 'neu', 'compound', 'story_com', 'music_com', 'direction_com',
       'actor_com', 'acting_com', 'story_pos', 'music_pos', 'direction_pos',
       'actor_pos', 'acting_pos', 'story_neg', 'music_neg', 'direction_neg',
       'actor_neg', 'acting_neg']]
y_data = temp.kor_audience

In [136]:
X_train, X_test, y_train, y_test =\
train_test_split(X_data, y_data,\
                    test_size = 0.3, \
                    random_state = 78)

In [137]:
DecisionTreeTest(X_train, X_test, y_train, y_test)
RandomForestTest(X_train, X_test, y_train, y_test)
GBMTest(X_train, X_test, y_train, y_test)
MLPClassifierTest(X_train, X_test, y_train, y_test)
logisticRegression(X_train, X_test, y_train, y_test)
LightGBMTest(X_train, X_test, y_train, y_test)

DecisionTreeTest 정확도 : 0.6361
랜덤포레스트 정확도 : 0.6792
GradientBoostingClassifier 정확도:0.7035
MLPClassifier 정확도:0.5553
로지스틱 회귀 값 : 0.522911
LGBMClassifier 정확도:0.6954


## 스토리 데이터로만

In [103]:
X_data = temp[['x0', 'x1', 'x2', 'x3', 'x4', 'label',
       'center_angle1', 'center_angle2', 'center_angle3', 'center_angle4',
       'center_angle5', 'center_angle6', 'center_angle7', 'center_angle8',
       'center_angle9', 'center_angle10']]
y_data = temp.kor_audience

In [104]:
X_train, X_test, y_train, y_test =\
train_test_split(X_data, y_data,\
                    test_size = 0.3, \
                    random_state = 78)

In [105]:
DecisionTreeTest(X_train, X_test, y_train, y_test)
RandomForestTest(X_train, X_test, y_train, y_test)
GBMTest(X_train, X_test, y_train, y_test)
MLPClassifierTest(X_train, X_test, y_train, y_test)
logisticRegression(X_train, X_test, y_train, y_test)
LightGBMTest(X_train, X_test, y_train, y_test)

DecisionTreeTest 정확도 : 0.4367
랜덤포레스트 정확도 : 0.5175
GradientBoostingClassifier 정확도:0.4933
MLPClassifier 정확도:0.5256
로지스틱 회귀 값 : 0.512129
LGBMClassifier 정확도:0.5148


## 리뷰, 스토리 빼고

In [106]:
X_data = temp[['showtm', 'prdtyear', 'domestic', 'mpaa',
       'raters', 'ratings', 'sf', 'family',
       'performance', 'horror', 'etc', 'documentary', 'drama', 'romance',
       'musical', 'mystery', 'crime', 'history', 'western', 'adult',
       'thriller', 'animation', 'action', 'adventure', 'war', 'comedy',
       'fantasy', 'director_appearance', 'director_revenue',
       'distributor_share', 'opendt_quarter', 'year_gap', 'showtypes_num',
       'actor_score']]
y_data = temp.kor_audience

In [107]:
X_train, X_test, y_train, y_test =\
train_test_split(X_data, y_data,\
                    test_size = 0.3, \
                    random_state = 78)

In [108]:
DecisionTreeTest(X_train, X_test, y_train, y_test)
RandomForestTest(X_train, X_test, y_train, y_test)
GBMTest(X_train, X_test, y_train, y_test)
MLPClassifierTest(X_train, X_test, y_train, y_test)
logisticRegression(X_train, X_test, y_train, y_test)
LightGBMTest(X_train, X_test, y_train, y_test)

DecisionTreeTest 정확도 : 0.6523
랜덤포레스트 정확도 : 0.7278
GradientBoostingClassifier 정확도:0.7062
MLPClassifier 정확도:0.5472
로지스틱 회귀 값 : 0.522911
LGBMClassifier 정확도:0.6954


## 리뷰만 빼고

In [109]:
X_data = temp[['showtm', 'prdtyear', 'domestic', 'mpaa', 'raters',
       'ratings', 'sf', 'family', 'performance',
       'horror', 'etc', 'documentary', 'drama', 'romance', 'musical',
       'mystery', 'crime', 'history', 'western', 'adult', 'thriller',
       'animation', 'action', 'adventure', 'war', 'comedy', 'fantasy',
       'director_appearance', 'director_revenue', 'distributor_share',
       'opendt_quarter', 'year_gap', 'showtypes_num', 'actor_score', 'x0', 'x1', 'x2', 'x3', 'x4', 'label',
       'center_angle1', 'center_angle2', 'center_angle3', 'center_angle4',
       'center_angle5', 'center_angle6', 'center_angle7', 'center_angle8',
       'center_angle9', 'center_angle10']]
y_data = temp.kor_audience

In [110]:
X_train, X_test, y_train, y_test =\
train_test_split(X_data, y_data,\
                    test_size = 0.3, \
                    random_state = 78)

In [111]:
DecisionTreeTest(X_train, X_test, y_train, y_test)
RandomForestTest(X_train, X_test, y_train, y_test)
GBMTest(X_train, X_test, y_train, y_test)
MLPClassifierTest(X_train, X_test, y_train, y_test)
logisticRegression(X_train, X_test, y_train, y_test)
LightGBMTest(X_train, X_test, y_train, y_test)

DecisionTreeTest 정확도 : 0.6146
랜덤포레스트 정확도 : 0.7170
GradientBoostingClassifier 정확도:0.7224
MLPClassifier 정확도:0.3369
로지스틱 회귀 값 : 0.522911
LGBMClassifier 정확도:0.7224


## 리뷰 null 제거

In [112]:
temp1 = temp[['showtm', 'prdtyear', 'domestic', 'mpaa', 'raters',
       'ratings', 'sf', 'family', 'performance',
       'horror', 'etc', 'documentary', 'drama', 'romance', 'musical',
       'mystery', 'crime', 'history', 'western', 'adult', 'thriller',
       'animation', 'action', 'adventure', 'war', 'comedy', 'fantasy',
       'director_appearance', 'director_revenue', 'distributor_share',
       'opendt_quarter', 'year_gap', 'showtypes_num', 'actor_score', 'neg',
       'pos', 'neu', 'compound', 'story_com', 'music_com', 'direction_com',
       'actor_com', 'acting_com', 'story_pos', 'music_pos', 'direction_pos',
       'actor_pos', 'acting_pos', 'story_neg', 'music_neg', 'direction_neg',
       'actor_neg', 'acting_neg', 'x0', 'x1', 'x2', 'x3', 'x4', 'label',
       'center_angle1', 'center_angle2', 'center_angle3', 'center_angle4',
       'center_angle5', 'center_angle6', 'center_angle7', 'center_angle8',
       'center_angle9', 'center_angle10', 'kor_audience']]

In [113]:
temp1 = temp1[temp1.compound != 0]

In [114]:
X_data = temp1.drop(['kor_audience'], axis = 1)
y_data = temp1.kor_audience

In [115]:
X_train, X_test, y_train, y_test =\
train_test_split(X_data, y_data,\
                    test_size = 0.3, \
                    random_state = 78)

In [116]:
DecisionTreeTest(X_train, X_test, y_train, y_test)
RandomForestTest(X_train, X_test, y_train, y_test)
GBMTest(X_train, X_test, y_train, y_test)
MLPClassifierTest(X_train, X_test, y_train, y_test)
logisticRegression(X_train, X_test, y_train, y_test)
LightGBMTest(X_train, X_test, y_train, y_test)

DecisionTreeTest 정확도 : 0.5876
랜덤포레스트 정확도 : 0.6981
GradientBoostingClassifier 정확도:0.7143
MLPClassifier 정확도:0.4636
로지스틱 회귀 값 : 0.530997
LGBMClassifier 정확도:0.6954


## 리뷰 null 제거 + 리뷰 특성 빼고

In [117]:
X_data = temp1[['showtm', 'prdtyear', 'domestic', 'mpaa',
       'raters', 'ratings', 'sf', 'family',
       'performance', 'horror', 'etc', 'documentary', 'drama', 'romance',
       'musical', 'mystery', 'crime', 'history', 'western', 'adult',
       'thriller', 'animation', 'action', 'adventure', 'war', 'comedy',
       'fantasy', 'director_appearance', 'director_revenue',
       'distributor_share', 'opendt_quarter', 'year_gap', 'showtypes_num',
       'actor_score']]
y_data = temp1.kor_audience

In [118]:
X_train, X_test, y_train, y_test =\
train_test_split(X_data, y_data,\
                    test_size = 0.3, \
                    random_state = 78)

In [119]:
DecisionTreeTest(X_train, X_test, y_train, y_test)
RandomForestTest(X_train, X_test, y_train, y_test)
GBMTest(X_train, X_test, y_train, y_test)
MLPClassifierTest(X_train, X_test, y_train, y_test)
logisticRegression(X_train, X_test, y_train, y_test)
LightGBMTest(X_train, X_test, y_train, y_test)

DecisionTreeTest 정확도 : 0.6712
랜덤포레스트 정확도 : 0.7278
GradientBoostingClassifier 정확도:0.7062
MLPClassifier 정확도:0.3585
로지스틱 회귀 값 : 0.522911
LGBMClassifier 정확도:0.6954


## 리뷰 null 제거 + 리뷰 특성 포함

In [120]:
X_data = temp1[['showtm', 'prdtyear', 'domestic', 'mpaa',
       'raters', 'ratings', 'sf', 'family',
       'performance', 'horror', 'etc', 'documentary', 'drama', 'romance',
       'musical', 'mystery', 'crime', 'history', 'western', 'adult',
       'thriller', 'animation', 'action', 'adventure', 'war', 'comedy',
       'fantasy', 'director_appearance', 'director_revenue',
       'distributor_share', 'opendt_quarter', 'year_gap', 'showtypes_num',
       'actor_score', 'neg', 'pos', 'neu', 'compound', 'story_com',
       'music_com', 'direction_com', 'actor_com', 'acting_com', 'story_pos',
       'music_pos', 'direction_pos', 'actor_pos', 'acting_pos', 'story_neg',
       'music_neg', 'direction_neg', 'actor_neg', 'acting_neg']]
y_data = temp1.kor_audience

In [121]:
X_train, X_test, y_train, y_test =\
train_test_split(X_data, y_data,\
                    test_size = 0.3, \
                    random_state = 78)

In [122]:
DecisionTreeTest(X_train, X_test, y_train, y_test)
RandomForestTest(X_train, X_test, y_train, y_test)
GBMTest(X_train, X_test, y_train, y_test)
MLPClassifierTest(X_train, X_test, y_train, y_test)
logisticRegression(X_train, X_test, y_train, y_test)
LightGBMTest(X_train, X_test, y_train, y_test)

DecisionTreeTest 정확도 : 0.6199
랜덤포레스트 정확도 : 0.6792
GradientBoostingClassifier 정확도:0.7035
MLPClassifier 정확도:0.5013
로지스틱 회귀 값 : 0.522911
LGBMClassifier 정확도:0.6954


## 스토리 + 댓글로만

In [123]:
X_data = temp[['neg',
       'pos', 'neu', 'compound', 'story_com', 'music_com', 'direction_com',
       'actor_com', 'acting_com', 'story_pos', 'music_pos', 'direction_pos',
       'actor_pos', 'acting_pos', 'story_neg', 'music_neg', 'direction_neg',
       'actor_neg', 'acting_neg', 'x0', 'x1', 'x2', 'x3', 'x4', 'label',
       'center_angle1', 'center_angle2', 'center_angle3', 'center_angle4',
       'center_angle5', 'center_angle6', 'center_angle7', 'center_angle8',
       'center_angle9', 'center_angle10']]
y_data = temp.kor_audience

In [124]:
X_train, X_test, y_train, y_test =\
train_test_split(X_data, y_data,\
                    test_size = 0.3, \
                    random_state = 78)

In [125]:
DecisionTreeTest(X_train, X_test, y_train, y_test)
RandomForestTest(X_train, X_test, y_train, y_test)
GBMTest(X_train, X_test, y_train, y_test)
MLPClassifierTest(X_train, X_test, y_train, y_test)
logisticRegression(X_train, X_test, y_train, y_test)
LightGBMTest(X_train, X_test, y_train, y_test)

DecisionTreeTest 정확도 : 0.4744
랜덤포레스트 정확도 : 0.5364
GradientBoostingClassifier 정확도:0.5391
MLPClassifier 정확도:0.5256
로지스틱 회귀 값 : 0.495957
LGBMClassifier 정확도:0.5337


## 스토리로만..

In [126]:
X_data = temp[['x0', 'x1', 'x2', 'x3', 'x4', 'label',
       'center_angle1', 'center_angle2', 'center_angle3', 'center_angle4',
       'center_angle5', 'center_angle6', 'center_angle7', 'center_angle8',
       'center_angle9', 'center_angle10']]
y_data = temp.kor_audience

In [127]:
X_train, X_test, y_train, y_test =\
train_test_split(X_data, y_data,\
                    test_size = 0.3, \
                    random_state = 78)

In [128]:
DecisionTreeTest(X_train, X_test, y_train, y_test)
RandomForestTest(X_train, X_test, y_train, y_test)
GBMTest(X_train, X_test, y_train, y_test)
MLPClassifierTest(X_train, X_test, y_train, y_test)
logisticRegression(X_train, X_test, y_train, y_test)
LightGBMTest(X_train, X_test, y_train, y_test)

DecisionTreeTest 정확도 : 0.4232
랜덤포레스트 정확도 : 0.5175
GradientBoostingClassifier 정확도:0.4933
MLPClassifier 정확도:0.5256
로지스틱 회귀 값 : 0.512129
LGBMClassifier 정확도:0.5148


# 데이터 뿔려보기

In [129]:
X_data = temp[['showtm', 'prdtyear', 'domestic', 'mpaa', 'raters',
       'ratings', 'sf', 'family', 'performance',
       'horror', 'etc', 'documentary', 'drama', 'romance', 'musical',
       'mystery', 'crime', 'history', 'western', 'adult', 'thriller',
       'animation', 'action', 'adventure', 'war', 'comedy', 'fantasy',
       'director_appearance', 'director_revenue', 'distributor_share',
       'opendt_quarter', 'year_gap', 'showtypes_num', 'actor_score', 'neg',
       'pos', 'neu', 'compound', 'story_com', 'music_com', 'direction_com',
       'actor_com', 'acting_com', 'story_pos', 'music_pos', 'direction_pos',
       'actor_pos', 'acting_pos', 'story_neg', 'music_neg', 'direction_neg',
       'actor_neg', 'acting_neg', 'x0', 'x1', 'x2', 'x3', 'x4', 'label',
       'center_angle1', 'center_angle2', 'center_angle3', 'center_angle4',
       'center_angle5', 'center_angle6', 'center_angle7', 'center_angle8',
       'center_angle9', 'center_angle10']]
y_data = temp.kor_audience

In [130]:
X_train, X_test, y_train, y_test =\
train_test_split(X_data, y_data,\
                    test_size = 0.3, \
                    random_state = 20)

In [131]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=0)
X_train_over, y_train_over = smote.fit_sample(X_train,y_train)

ValueError: Expected n_neighbors <= n_samples,  but n_samples = 4, n_neighbors = 6

In [ ]:
DecisionTreeTest(X_train_over, X_test, y_train_over, y_test)
RandomForestTest(X_train_over, X_test, y_train_over, y_test)
GBMTest(X_train_over, X_test, y_train_over, y_test)
MLPClassifierTest(X_train_over, X_test, y_train_over, y_test)
logisticRegression(X_train_over, X_test, y_train_over, y_test)
LightGBMTest(X_train_over, X_test, y_train_over, y_test)